# NC coordinates data from json 

In [17]:
import requests

# Fetch the content of the URL
response = requests.get("https://raw.githubusercontent.com/hackforla/311-data/main/data/nc-boundary-2019-modified.json")

# Check if the request was successful
if response.status_code == 200:
    # Save the data to a file
    with open("nc-boundary-2019-modified.json", "w") as file:
        file.write(response.text)
    print("Data saved successfully.")
else:
    print("Failed to fetch data:", response.status_code)

Data saved successfully.


In [18]:
# Load the JSON data
with open('nc-boundary-2019-modified.json') as json_file:
    data = json.load(json_file)

# Create a list to hold the coordinates
coordinates_list = []

# Iterate over each feature
for feature in data['features']:
    # Extract coordinates
    geometry = feature['geometry']
    coordinates = geometry['coordinates']

    # Append coordinates to the list
    coordinates_list.append(coordinates)

# Save the coordinates to a JSON file
with open('coordinates.json', 'w') as json_output_file:
    json.dump(coordinates_list, json_output_file, indent=4)


## Filtering Data Points Based on Polygon Coordinates

In [19]:
# Importing the necessary library 
import requests
import pandas as pd
import json
import csv

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings  

# Ignore warnings
warnings.filterwarnings('ignore')

In [20]:
output_file = "./Data_csvfiles/2024.csv"

In [21]:
data=pd.read_csv(output_file)

In [22]:
# Display the first few rows of the dataset for initial examination

data.head()

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,Location,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct
0,1-4601720641,1/1/2024 0:00,1/3/2024 10:37,SR Created,LASAN,Bulky Items,Closed,Call,LASAN,NaN,...,POINT (-118.342143549 34.0211832597),673,D,1,South Los Angeles APC,10,Heather Hutt,75.0,West Adams,SOUTHWEST
1,1-4601721601,1/1/2024 0:05,1/1/2024 0:05,SR Created,BSL,Multiple Streetlight Issue,Open,Self Service,Self Service,NaN,...,POINT (-118.3036039481 33.789784475205),793,J,5,Harbor APC,15,Tim McOsker,92.0,Harbor City,HARBOR
2,1-4601723091,1/1/2024 0:06,1/1/2024 0:06,SR Created,BSL,Single Streetlight Issue,Open,Mobile App,Self Service,iOS,...,POINT (-118.36056718 34.0858735),593,B,6,Central APC,5,Katy Yaroslavsky,58.0,Mid-City West,WILSHIRE
3,1-4601721751,1/1/2024 0:11,1/2/2024 19:01,SR Created,LASAN,Bulky Items,Closed,Mobile App,Self Service,Android,...,POINT (-118.533785604 34.2067120073),530,J,4,South Valley APC,4,Nithya Raman,15.0,Reseda,WEST VALLEY
4,1-4601723281,1/1/2024 0:18,1/29/2024 9:06,SR Created,OCB,Graffiti Removal,Closed,Mobile App,Self Service,iOS,...,POINT (-118.28067055 34.0423584867),634,B,5,Central APC,1,Eunisses Hernandez,76.0,Pico Union,RAMPART


In [23]:
data.columns

Index(['SRNumber', 'CreatedDate', 'UpdatedDate', 'ActionTaken', 'Owner',
       'RequestType', 'Status', 'RequestSource', 'CreatedByUserOrganization',
       'MobileOS', 'Anonymous', 'AssignTo', 'ServiceDate', 'ClosedDate',
       'AddressVerified', 'ApproximateAddress', 'Address', 'HouseNumber',
       'Direction', 'StreetName', 'Suffix', 'ZipCode', 'Latitude', 'Longitude',
       'Location', 'TBMPage', 'TBMColumn', 'TBMRow', 'APC', 'CD', 'CDMember',
       'NC', 'NCName', 'PolicePrecinct'],
      dtype='object')

In [24]:
data.shape

(2148, 34)

In [25]:
df=data.copy()

In [26]:
df.shape

(2148, 34)

In [27]:
import json
from shapely.geometry import Point, Polygon, MultiPolygon
import pandas as pd
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)  # Set the logging level to INFO

# Load the JSON structure from the file
with open('data.json', 'r') as f:
    nc_data = json.load(f)

# Define the is_within_polygon function with logging
def is_within_polygon(lat, lon, nc_data):
    point = Point(lon, lat)
    for feature in nc_data:
        geometry_type = feature['geometry_type']
        coordinates = feature['coordinates']
        if geometry_type == 'MultiPolygon':
            multipolygon = MultiPolygon([Polygon(polygon[0]) for polygon in coordinates])
            if multipolygon.contains(point):
                logging.info(f"Point ({lat}, {lon}) is within {feature['name']}")
                return True
        elif geometry_type == 'Polygon':
            polygon = Polygon(coordinates[0])
            if polygon.contains(point):
                logging.info(f"Point ({lat}, {lon}) is within {feature['name']}")
                return True
    return False

# Load your DataFrame
#df = pd.read_csv('your_dataframe.csv')  # Replace 'your_dataframe.csv' with your actual file path

# Print initial number of rows
print("Initial number of rows:", len(df))

# Drop rows with null values in longitude and latitude columns
dropped_rows = len(df) - len(df.dropna(subset=['Longitude', 'Latitude']))
print("Dropped", dropped_rows, "rows due to null values in Longitude or Latitude columns.")

df.dropna(subset=['Longitude', 'Latitude'], inplace=True)
df_new = df.drop_duplicates(subset=['Longitude', 'Latitude'])






Initial number of rows: 2148
Dropped 0 rows due to null values in Longitude or Latitude columns.


In [28]:
df.shape

(2148, 34)

In [29]:
df_new.shape

(1864, 34)

In [30]:
# Filter DataFrame based on polygon coordinates using the is_within_polygon function
filtered_mask = df.apply(lambda row: is_within_polygon(row['Latitude'], row['Longitude'], nc_data), axis=1)
filtered_df = df[filtered_mask]

# Print number of rows after filtering
print("Number of rows after filtering:", len(filtered_df))

INFO:root:Point (34.02118326, -118.3421435) is within WEST ADAMS NC


INFO:root:Point (33.78978448, -118.3036039) is within HARBOR CITY NC
INFO:root:Point (34.0858735, -118.3605672) is within MID CITY WEST CC
INFO:root:Point (34.20671201, -118.5337856) is within RESEDA NC
INFO:root:Point (34.04235849, -118.2806706) is within PICO UNION NC
INFO:root:Point (34.27864794, -118.3912721) is within FOOTHILL TRAILS DISTRICT NC
INFO:root:Point (34.09793511, -118.3058986) is within EAST HOLLYWOOD NC
INFO:root:Point (34.21754139, -118.4686036) is within NORTH HILLS EAST
INFO:root:Point (34.09826713, -118.3060253) is within EAST HOLLYWOOD NC
INFO:root:Point (34.09939647, -118.3058383) is within EAST HOLLYWOOD NC
INFO:root:Point (34.09993968, -118.3059727) is within EAST HOLLYWOOD NC
INFO:root:Point (34.09993968, -118.3059727) is within EAST HOLLYWOOD NC
INFO:root:Point (34.09789271, -118.2996018) is within EAST HOLLYWOOD NC
INFO:root:Point (34.10156893, -118.3095182) is within HOLLYWOOD STUDIO DISTRICT NC
INFO:root:Point (34.05180924, -118.427677) is within WESTSIDE

KeyboardInterrupt: 

In [ ]:
filtered_df.head(2)

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,Location,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct
0,1-4601720641,1/1/2024 0:00,1/3/2024 10:37,SR Created,LASAN,Bulky Items,Closed,Call,LASAN,NaN,...,POINT (-118.342143549 34.0211832597),673,D,1,South Los Angeles APC,10,Heather Hutt,75.0,West Adams,SOUTHWEST
1,1-4601721601,1/1/2024 0:05,1/1/2024 0:05,SR Created,BSL,Multiple Streetlight Issue,Open,Self Service,Self Service,NaN,...,POINT (-118.3036039481 33.789784475205),793,J,5,Harbor APC,15,Tim McOsker,92.0,Harbor City,HARBOR


In [ ]:
# Extract omitted data points
omitted_df = df[~filtered_mask]

print("Omitted data points:")
omitted_df.head()

Omitted data points:


,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,Location,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct
73,1-4601754011,1/1/2024 7:21,1/2/2024 17:12,SR Created,LASAN,Bulky Items,Closed,Mobile App,Self Service,Android,...,POINT (-118.197563358 34.0599350461),635,C,2,East Los Angeles APC,14,Kevin de León,NaN,NaN,HOLLENBECK
177,1-4601770621,1/1/2024 9:02,1/5/2024 9:43,SR Created,OCB,Graffiti Removal,Closed,Mobile App,Self Service,iOS,...,POINT (-118.196652416 34.0578876265),635,C,3,East Los Angeles APC,14,Kevin de León,NaN,NaN,HOLLENBECK
218,1-4601774861,1/1/2024 9:24,1/3/2024 12:16,SR Created,OCB,Graffiti Removal,Closed,Mobile App,Self Service,iOS,...,POINT (-118.196652416 34.0578876265),635,C,3,East Los Angeles APC,14,Kevin de León,NaN,NaN,HOLLENBECK
227,1-4601777841,1/1/2024 9:33,1/5/2024 9:43,SR Created,OCB,Graffiti Removal,Closed,Mobile App,Self Service,iOS,...,POINT (-118.197047194 34.0577005001),635,C,3,East Los Angeles APC,14,Kevin de León,NaN,NaN,HOLLENBECK
232,1-4601780511,1/1/2024 9:35,1/6/2024 8:25,SR Created,OCB,Graffiti Removal,Closed,Mobile App,Self Service,iOS,...,POINT (-118.199533243 34.059196822),635,C,3,East Los Angeles APC,14,Kevin de León,NaN,NaN,HOLLENBECK


In [ ]:
filtered_df.shape

(2087, 34)

In [ ]:
omitted_df.shape

(61, 34)

# Geopandas 
## Filtering DataFrame Points within GeoDataFrame Polygons

### This code snippet filters points from a DataFrame to include only those that fall within the polygons of a GeoDataFrame.


In [31]:
import geopandas as gpd
from shapely.geometry import Point

# Load the GeoDataFrame from the GeoJSON file
gdf = gpd.read_file('nc-boundary-2019-modified.json')




In [32]:
gdf.head()

,name,waddress,certified,objectid,service_re,nc_id,dwebsite,geometry
0,LINCOLN HEIGHTS NC,http://www.lincolnheightsnc.org/,2002-04-27 00:00:00+00:00,1,REGION 8 - NORTH EAST LA,47,http://empowerla.org/LHNC,"MULTIPOLYGON (((-118.20503 34.05558, -118.2060..."
1,VOICES OF 90037,None,2008-06-16 00:00:00+00:00,2,REGION 9 - SOUTH LA 2,109,http://empowerla.org/VOICESOF90037,"MULTIPOLYGON (((-118.28273 34.01114, -118.2823..."
2,LOS FELIZ NC,http://www.losfeliznc.org/,2002-06-11 00:00:00+00:00,3,REGION 7 - EAST,36,http://empowerla.org/LFNC,"MULTIPOLYGON (((-118.30027 34.15793, -118.2988..."
3,MISSION HILLS NC,http://mhnconline.org/,2005-04-19 00:00:00+00:00,4,REGION 1 - NORTH EAST VALLEY,101,http://empowerla.org/MHNC,"MULTIPOLYGON (((-118.46933 34.28050, -118.4693..."
4,GREATER TOLUCA LAKE NC,http://www.gtlnc.org/,2002-09-10 00:00:00+00:00,5,REGION 4 - SOUTH EAST VALLEY,28,http://empowerla.org/GTLNC,"MULTIPOLYGON (((-118.35520 34.16491, -118.3552..."


In [33]:
output_file = "./Data_csvfiles/2024_original.csv"

In [34]:
data=pd.read_csv(output_file)

In [35]:
df=data.copy()

In [44]:
df.shape

(101146, 34)

In [36]:
# Convert DataFrame to GeoDataFrame with Point geometries
geometry = [Point(lon, lat) for lon, lat in zip(df['Longitude'], df['Latitude'])]
gdf_points = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')


In [37]:
gdf_points.shape

(101146, 35)

In [38]:
# Perform a spatial join to check if points fall within polygons
merged_gdf = gpd.sjoin(gdf_points, gdf, how='inner', op='within')

merged_gdf.head()

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,PolicePrecinct,geometry,index_right,name,waddress,certified,objectid,service_re,nc_id,dwebsite
0,1-4601720641,1/1/2024 0:00,1/3/2024 10:37,SR Created,LASAN,Bulky Items,Closed,Call,LASAN,NaN,...,SOUTHWEST,POINT (-118.34214 34.02118),11,WEST ADAMS NC,http://www.westadamsnc.org/,2002-02-19 00:00:00+00:00,17,REGION 10 - SOUTH LA 1,75,http://empowerla.org/WANC
230,1-4601782121,1/1/2024 9:35,1/2/2024 13:20,SR Created,LASAN,Bulky Items,Closed,Call,LASAN,NaN,...,SOUTHWEST,POINT (-118.33846 34.01958),11,WEST ADAMS NC,http://www.westadamsnc.org/,2002-02-19 00:00:00+00:00,17,REGION 10 - SOUTH LA 1,75,http://empowerla.org/WANC
243,1-4601782941,1/1/2024 9:37,1/2/2024 11:42,SR Created,LASAN,Metal/Household Appliances,Closed,Call,LASAN,NaN,...,SOUTHWEST,POINT (-118.33846 34.01958),11,WEST ADAMS NC,http://www.westadamsnc.org/,2002-02-19 00:00:00+00:00,17,REGION 10 - SOUTH LA 1,75,http://empowerla.org/WANC
249,1-4601784081,1/1/2024 9:39,1/2/2024 13:46,SR Created,LASAN,Electronic Waste,Closed,Call,LASAN,NaN,...,SOUTHWEST,POINT (-118.33846 34.01958),11,WEST ADAMS NC,http://www.westadamsnc.org/,2002-02-19 00:00:00+00:00,17,REGION 10 - SOUTH LA 1,75,http://empowerla.org/WANC
360,1-4601806271,1/1/2024 10:27,1/6/2024 16:15,SR Created,LASAN,Bulky Items,Closed,Self Service,Self Service,NaN,...,SOUTHWEST,POINT (-118.36654 34.03335),11,WEST ADAMS NC,http://www.westadamsnc.org/,2002-02-19 00:00:00+00:00,17,REGION 10 - SOUTH LA 1,75,http://empowerla.org/WANC


In [39]:
merged_gdf.columns

Index(['SRNumber', 'CreatedDate', 'UpdatedDate', 'ActionTaken', 'Owner',
       'RequestType', 'Status', 'RequestSource', 'CreatedByUserOrganization',
       'MobileOS', 'Anonymous', 'AssignTo', 'ServiceDate', 'ClosedDate',
       'AddressVerified', 'ApproximateAddress', 'Address', 'HouseNumber',
       'Direction', 'StreetName', 'Suffix', 'ZipCode', 'Latitude', 'Longitude',
       'Location', 'TBMPage', 'TBMColumn', 'TBMRow', 'APC', 'CD', 'CDMember',
       'NC', 'NCName', 'PolicePrecinct', 'geometry', 'index_right', 'name',
       'waddress', 'certified', 'objectid', 'service_re', 'nc_id', 'dwebsite'],
      dtype='object')

In [46]:
filtered_df = merged_gdf[df.columns]



In [47]:
filtered_df.columns

Index(['SRNumber', 'CreatedDate', 'UpdatedDate', 'ActionTaken', 'Owner',
       'RequestType', 'Status', 'RequestSource', 'CreatedByUserOrganization',
       'MobileOS', 'Anonymous', 'AssignTo', 'ServiceDate', 'ClosedDate',
       'AddressVerified', 'ApproximateAddress', 'Address', 'HouseNumber',
       'Direction', 'StreetName', 'Suffix', 'ZipCode', 'Latitude', 'Longitude',
       'Location', 'TBMPage', 'TBMColumn', 'TBMRow', 'APC', 'CD', 'CDMember',
       'NC', 'NCName', 'PolicePrecinct'],
      dtype='object')

In [45]:
df.shape

(101146, 34)

In [48]:
filtered_df.shape

(99142, 34)

In [50]:
filtered_df.tail()

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,Location,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct
95206,1-4630684311,1/24/2024 14:28,1/29/2024 13:35,SR Created,OCB,Graffiti Removal,Closed,Mobile App,Self Service,Android,...,POINT (-118.236758272 34.0532203785),634.0,G,3.0,Central APC,14.0,Kevin de León,46.0,Arts District Little Tokyo,CENTRAL
96196,1-4631003281,1/24/2024 17:36,1/31/2024 16:34,SR Created,LASAN,Bulky Items,Closed,Call,LASAN,NaN,...,POINT (-118.240413172 34.050693851),634.0,G,4.0,Central APC,14.0,Kevin de León,46.0,Arts District Little Tokyo,CENTRAL
96412,1-4631071721,1/24/2024 18:57,3/7/2024 11:22,SR Created,OCB,Graffiti Removal,Closed,Call,ITA,NaN,...,POINT (-118.234460703 34.0433555937),634.0,H,5.0,Central APC,14.0,Kevin de León,46.0,Arts District Little Tokyo,CENTRAL
96414,1-4631074721,1/24/2024 18:58,1/28/2024 9:25,SR Created,OCB,Graffiti Removal,Cancelled,Call,ITA,NaN,...,POINT (-118.242272848 34.0473633182),634.0,G,4.0,Central APC,14.0,Kevin de León,46.0,Arts District Little Tokyo,CENTRAL
97077,1-4631389411,1/25/2024 7:43,1/25/2024 7:45,SR Created,OCB,Graffiti Removal,Closed,Driver Self Report,OCB,NaN,...,POINT (-118.23929105 34.0469728524),634.0,G,4.0,Central APC,14.0,NaN,46.0,Arts District Little Tokyo,CENTRAL
